In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

/home/gaddafi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
PATH = os.getcwd()
numpy.random.seed(7)

In [3]:
PATH

'/home/gaddafi/SmartGrid'

In [4]:
df = pd.read_excel(PATH+'/data/final_data.xlsx')

In [5]:
df.head()

,Date,Hour,Market Demand,Ontario Demand,HOEP,BIOFUEL,GAS,HYDRO,NUCLEAR,SOLAR,WIND,TotalOutput,Company_HOEP
0,2018-01-01,1,18974,16627,51.29,3,1583,4152,11671,0,992,18401,310
1,2018-01-01,2,18447,16084,43.59,1,1192,4093,11672,0,1124,18082,27
2,2018-01-01,3,18453,15866,93.60,0,1040,4231,11672,0,1265,18208,87
3,2018-01-01,4,18662,15725,54.78,0,1041,3895,11669,0,1369,17974,65
4,2018-01-01,5,18060,15470,14.35,0,1004,3271,11674,0,1848,17797,194


In [6]:
data_X = df[['Ontario Demand', 'HOEP']]
dataset = data_X.values
dataset = dataset.astype('float32')

In [7]:
data_X.head()

,Ontario Demand,HOEP
0,16627,51.29
1,16084,43.59
2,15866,93.60
3,15725,54.78
4,15470,14.35


In [8]:
# normalize the dataset
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

In [9]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

5869 2891


In [10]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=48):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0:2]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [11]:
# reshape into X=t and Y=t+48
look_back = 48
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [12]:
trainX[0:49]

array([[[0.4792503 , 0.15047428],
        [0.436491  , 0.12966517],
        [0.4193244 , 0.26481634],
        ...,
        [0.6516261 , 0.12439531],
        [0.57098985, 0.0335108 ],
        [0.5024017 , 0.0921817 ]],

       [[0.436491  , 0.12966517],
        [0.4193244 , 0.26481634],
        [0.40822113, 0.15990594],
        ...,
        [0.57098985, 0.0335108 ],
        [0.5024017 , 0.0921817 ],
        [0.45168912, 0.01810664]],

       [[0.4193244 , 0.26481634],
        [0.40822113, 0.15990594],
        [0.3881408 , 0.05064454],
        ...,
        [0.5024017 , 0.0921817 ],
        [0.45168912, 0.01810664],
        [0.41688323, 0.06437316]],

       ...,

       [[0.57098985, 0.0335108 ],
        [0.5024017 , 0.0921817 ],
        [0.45168912, 0.01810664],
        ...,
        [0.74328685, 0.17106721],
        [0.7335223 , 0.17082398],
        [0.7195054 , 0.1766073 ]],

       [[0.5024017 , 0.0921817 ],
        [0.45168912, 0.01810664],
        [0.41688323, 0.06437316],
        .

In [13]:
trainY.shape

(5820,)

In [14]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
testX = numpy.reshape(testX, (testX.shape[0], trainX.shape[1], testX.shape[2]))

In [15]:
trainX.shape

(5820, 48, 2)

In [16]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 2)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
 - 110s - loss: 0.0092
Epoch 2/100
 - 103s - loss: 0.0012
Epoch 3/100
 - 100s - loss: 7.7590e-04
Epoch 4/100
 - 100s - loss: 6.7452e-04
Epoch 5/100
 - 100s - loss: 6.6615e-04
Epoch 6/100
 - 98s - loss: 6.5285e-04
Epoch 7/100
 - 98s - loss: 6.3516e-04
Epoch 8/100
 - 99s - loss: 6.2689e-04
Epoch 9/100
 - 98s - loss: 6.3098e-04
Epoch 10/100
 - 97s - loss: 6.1976e-04
Epoch 11/100
 - 98s - loss: 6.2218e-04
Epoch 12/100
 - 97s - loss: 6.1755e-04
Epoch 13/100
 - 96s - loss: 6.1687e-04
Epoch 14/100


KeyboardInterrupt: 

In [20]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainY = scaler.inverse_transform([trainY])
trainPredict = scaler.inverse_transform(trainPredict)

testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

ValueError: operands could not be broadcast together with shapes (1,5820) (2,) (1,5820) 

In [18]:
trainPredict.shape

(5820, 1)

In [19]:
trainY.shape

(5820,)